<a href="https://colab.research.google.com/github/FilipePessoa30/riot_api/blob/main/API_RIOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd

# Substitua pela sua chave de API
API_KEY = "RGAPI-1c2ba4e6-db6d-4f94-a475-b893ac699da7"

# Região do servidor
REGION = "br1"  # Escolha a região apropriada (br1, na1, euw1, etc.)

# Testar um endpoint diferente: Status do Servidor
URL_STATUS = f"https://{REGION}.api.riotgames.com/lol/status/v4/platform-data"

# Testar um endpoint diferente: Challenger Rank no modo Solo/Duo
URL_CHALLENGER = f"https://{REGION}.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5"

# Cabeçalhos com a API Key
headers = {
    "X-Riot-Token": API_KEY
}

# Fazer requisição para o status do servidor
response_status = requests.get(URL_STATUS, headers=headers)
print("--- Teste: Status do Servidor ---")
print(f"Status Code: {response_status.status_code}")
print(f"Resposta: {response_status.text}\n")

# Fazer requisição para obter os jogadores Challenger
response_challenger = requests.get(URL_CHALLENGER, headers=headers)
print("--- Teste: Top Challenger Players ---")
print(f"Status Code: {response_challenger.status_code}")
print(f"Resposta: {response_challenger.text}\n")

# Verificar se a resposta da API dos Challengers foi bem-sucedida
if response_challenger.status_code == 200:
    challenger_data = response_challenger.json()
    print("Dados dos jogadores Challenger:")
    print(pd.DataFrame(challenger_data.get("entries", [])))  # Exibir lista de jogadores em ranking Challenger
else:
    print("Erro ao buscar dados dos jogadores Challenger.")

--- Teste: Status do Servidor ---
Status Code: 200
Resposta: {"id":"BR1","name":"Brazil","locales":["ar_AE","cs_CZ","de_DE","el_GR","en_AU","en_GB","en_PH","en_SG","en_US","es_AR","es_ES","es_MX","fr_FR","hu_HU","it_IT","ja_JP","ko_KR","pl_PL","pt_BR","ro_RO","ru_RU","th_TH","tr_TR","vi_VN","zh_MY","zh_TW"],"maintenances":[{"id":10042,"created_at":"2025-01-17T02:12:00.422698+00:00","updated_at":null,"archive_at":null,"titles":[{"locale":"en_US","content":"Account Transfers Unavailable"},{"locale":"ar_AE","content":"عدم توفُّر عمليات نقل الحساب"},{"locale":"cs_CZ","content":"Převody účtu nejsou k dispozici"},{"locale":"de_DE","content":"Kontotransfer nicht verfügbar"},{"locale":"el_GR","content":"Οι μεταφορές λογαριασμών δεν είναι διαθέσιμες"},{"locale":"en_AU","content":"Account Transfers Unavailable"},{"locale":"en_GB","content":"Account Transfers Unavailable"},{"locale":"en_PH","content":"Account Transfers Unavailable"},{"locale":"en_SG","content":"Account Transfers Unavailable"},{"l

In [ ]:
import requests
import pandas as pd
import json

# Sua chave API Riot – certifique-se de usar uma chave válida e atual.
API_KEY = "RGAPI-c46c24fc-bd6b-40c2-856c-840eb0a91eea"

# Configurações de região e fila para League Challenger
REGION = "BR1"
QUEUE = "RANKED_SOLO_5x5"

def get_challenger_league(queue=QUEUE, region=REGION):
    """
    Consulta os dados da Challenger League para a fila especificada.
    Esse endpoint retorna uma lista de jogadores (entradas) na liga Challenger.
    """
    url = f"https://{region}.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/{queue}"
    headers = {"X-Riot-Token": API_KEY}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print("Erro ao obter dados da Challenger League:", response.text)
        return None

def get_data_dragon_champions(version="13.6.1", language="pt_BR"):
    """
    Puxa os dados estáticos de campeões do Data Dragon.
    Esses dados não requerem autenticação e trazem informações como estatísticas base, lore, etc.
    """
    url = f"http://ddragon.leagueoflegends.com/cdn/{version}/data/{language}/champion.json"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print("Erro ao obter dados do Data Dragon:", response.text)
        return None

def main():
    # 1. Obtendo dados da Challenger League
    challenger_data = get_challenger_league()
    if challenger_data:
        # O JSON retornado possui uma chave "entries" com os dados dos jogadores
        entries = challenger_data.get("entries", [])
        if entries:
            df_challenger = pd.DataFrame(entries)
            df_challenger.to_csv("challenger_league.csv", index=False, encoding="utf-8")
            print("Dados da Challenger League salvos em 'challenger_league.csv'.")
        else:
            print("Nenhuma entrada encontrada na Challenger League.")

    # 2. Obtendo dados dos Campeões via Data Dragon
    champions_data = get_data_dragon_champions()
    if champions_data:
        # Os dados dos campeões estão em um dicionário sob a chave "data"
        champions = champions_data.get("data", {})
        champions_list = list(champions.values())
        if champions_list:
            df_champions = pd.DataFrame(champions_list)
            df_champions.to_csv("champions_data.csv", index=False, encoding="utf-8")
            print("Dados dos campeões salvos em 'champions_data.csv'.")
        else:
            print("Nenhum dado de campeão encontrado.")

if __name__ == "__main__":
    main()


Dados da Challenger League salvos em 'challenger_league.csv'.
Dados dos campeões salvos em 'champions_data.csv'.


In [ ]:
import requests
import pandas as pd
import time

API_KEY = "RGAPI-c46c24fc-bd6b-40c2-856c-840eb0a91eea"  # 🔑 Substitua pela sua chave
REGION = 'americas'  # Região da API de partidas
REGION_SUMMONER = 'br1'  # Região do summoner

# 1. Buscar partidas recentes de um jogador de elite (exemplo: Challenger)
def get_challenger_matches(api_key, num_matches=50):
    # Passo 1: Obter PUUID de um jogador Challenger
    challenger_url = f'https://{REGION_SUMMONER}.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5'
    response = requests.get(challenger_url, headers={'X-Riot-Token': api_key})
    summoner_id = response.json()['entries'][0]['summonerId']  # Primeiro jogador da lista

    # Passo 2: Obter PUUID do summoner
    summoner_url = f'https://{REGION_SUMMONER}.api.riotgames.com/lol/summoner/v4/summoners/{summoner_id}'
    response = requests.get(summoner_url, headers={'X-Riot-Token': api_key})
    puuid = response.json()['puuid']

    # Passo 3: Buscar IDs das partidas recentes
    matches_url = f'https://{REGION}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?type=ranked&count={num_matches}'
    match_ids = requests.get(matches_url, headers={'X-Riot-Token': api_key}).json()

    return match_ids

# 2. Coletar dados das partidas e processar estatísticas
def get_champion_stats(match_ids, api_key):
    champion_data = {}

    for match_id in match_ids:
        # Buscar dados da partida
        match_url = f'https://{REGION}.api.riotgames.com/lol/match/v5/matches/{match_id}'
        response = requests.get(match_url, headers={'X-Riot-Token': api_key})

        if response.status_code != 200:
            continue

        match_data = response.json()
        map_id = match_data['info']['mapId']  # ID do mapa

        for participant in match_data['info']['participants']:
            champion = participant['championName']
            stats = participant

            # Inicializar dados do campeão no mapa
            if champion not in champion_data:
                champion_data[champion] = {}

            if map_id not in champion_data[champion]:
                champion_data[champion][map_id] = {
                    'games': 0,
                    'kills': 0,
                    'deaths': 0,
                    'assists': 0,
                    'gold': 0,
                    'damage': 0,
                    'wins': 0
                }

            # Atualizar estatísticas
            data = champion_data[champion][map_id]
            data['games'] += 1
            data['kills'] += stats['kills']
            data['deaths'] += stats['deaths']
            data['assists'] += stats['assists']
            data['gold'] += stats['goldEarned']
            data['damage'] += stats['totalDamageDealtToChampions']
            data['wins'] += 1 if stats['win'] else 0

        time.sleep(1.2)  # Respeitar rate limits (100 requisições/2 minutos)

    return champion_data

# 3. Gerar relatório
def generate_report(champion_data):
    report = []

    for champion, maps in champion_data.items():
        for map_id, stats in maps.items():
            avg_kills = stats['kills'] / stats['games']
            avg_deaths = stats['deaths'] / stats['games']
            avg_assists = stats['assists'] / stats['games']
            avg_gold = stats['gold'] / stats['games']
            avg_damage = stats['damage'] / stats['games']
            win_rate = (stats['wins'] / stats['games']) * 100

            report.append({
                'Champion': champion,
                'Map': map_id,
                'Jogos': stats['games'],
                'Win Rate (%)': round(win_rate, 2),
                'Kills (avg)': round(avg_kills, 2),
                'Deaths (avg)': round(avg_deaths, 2),
                'Assists (avg)': round(avg_assists, 2),
                'KDA': round((avg_kills + avg_assists) / avg_deaths, 2) if avg_deaths > 0 else 'Perfect',
                'Gold (avg)': round(avg_gold, 2),
                'Dano (avg)': round(avg_damage, 2)
            })

    return pd.DataFrame(report)

# Executar
if __name__ == '__main__':
    # Coletar 50 partidas de um jogador Challenger
    match_ids = get_challenger_matches(API_KEY, num_matches=50)

    # Processar estatísticas
    stats = get_champion_stats(match_ids, API_KEY)

    # Gerar relatório
    df = generate_report(stats)

    # Ordenar por Win Rate e Dano
    df_sorted = df.sort_values(by=['Win Rate (%)', 'Dano (avg)'], ascending=False)

    # Exibir resultados
    print(df_sorted.head(20))

    Champion  Map  Jogos  Win Rate (%)  Kills (avg)  Deaths (avg)  \
87    Irelia   11      4         100.0          8.5          6.50   
108      Zoe   11      1         100.0          7.0          4.00   
49   Orianna   11      1         100.0          2.0          5.00   
101     Fizz   11      1         100.0         11.0          3.00   
112   Lillia   11      2         100.0         10.5          1.00   
109     Olaf   11      1         100.0          8.0          2.00   
111      Jax   11      1         100.0          7.0          8.00   
97   Sejuani   11      1         100.0          8.0          6.00   
71    Aatrox   11      1         100.0          1.0          8.00   
40    Rengar   11      6         100.0          9.5          4.33   
77    Veigar   11      1         100.0          6.0          3.00   
113    Sivir   11      1         100.0          4.0          2.00   
50     Ivern   11      2         100.0          4.5          1.50   
55      Ekko   11      1         1

In [ ]:
import requests
import pandas as pd
import time

API_KEY = "RGAPI-c46c24fc-bd6b-40c2-856c-840eb0a91eea"
REGION = 'americas'
REGION_SUMMONER = 'br1'

# 1. Buscar múltiplos jogadores Challenger
def get_challenger_players(api_key, num_players=10):
    challenger_url = f'https://{REGION_SUMMONER}.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5'
    response = requests.get(challenger_url, headers={'X-Riot-Token': api_key})
    data = response.json()

    # Verifica se a requisição foi bem-sucedida
    if response.status_code != 200:
        print(f"Erro ao obter dados da Challenger League (status {response.status_code}):")
        print(data)
        return []

    # Verifica se a chave 'entries' está presente na resposta
    if 'entries' not in data:
        print("Resposta inesperada da API:", data)
        return []

    return [entry['summonerId'] for entry in data['entries'][:num_players]]


# 2. Coletar partidas de vários jogadores
def get_matches_from_players(summoner_ids, api_key, matches_per_player=10):
    all_match_ids = []

    for summoner_id in summoner_ids:
        # Obter PUUID do summoner
        summoner_url = f'https://{REGION_SUMMONER}.api.riotgames.com/lol/summoner/v4/summoners/{summoner_id}'
        response = requests.get(summoner_url, headers={'X-Riot-Token': api_key})
        puuid = response.json()['puuid']

        # Buscar IDs das partidas
        matches_url = f'https://{REGION}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?type=ranked&count={matches_per_player}'
        match_ids = requests.get(matches_url, headers={'X-Riot-Token': api_key}).json()
        all_match_ids.extend(match_ids)

        time.sleep(1.2)

    return list(set(all_match_ids))  # Remover partidas duplicadas

# 3. Processar estatísticas com filtro
def generate_report(champion_data, min_games=5):
    report = []

    for champion, maps in champion_data.items():
        for map_id, stats in maps.items():
            if stats['games'] < min_games:
                continue  # Ignorar dados insignificantes
            avg_kills = stats['kills'] / stats['games']
            avg_deaths = stats['deaths'] / stats['games']
            avg_assists = stats['assists'] / stats['games']
            avg_gold = stats['gold'] / stats['games']
            avg_damage = stats['damage'] / stats['games']
            win_rate = (stats['wins'] / stats['games']) * 100

            report.append({
                'Champion': champion,
                'Map': map_id,
                'Jogos': stats['games'],
                'Win Rate (%)': round(win_rate, 2),
                'Kills (avg)': round(avg_kills, 2),
                'Deaths (avg)': round(avg_deaths, 2),
                'Assists (avg)': round(avg_assists, 2),
                'KDA': round((avg_kills + avg_assists) / avg_deaths, 2) if avg_deaths > 0 else 'Perfect',
                'Gold (avg)': round(avg_gold, 2),
                'Dano (avg)': round(avg_damage, 2)
            })


    return pd.DataFrame(report)

# Execução principal
if __name__ == '__main__':
    # Coletar 10 jogadores Challenger e 10 partidas cada
    summoner_ids = get_challenger_players(API_KEY, num_players=10)
    match_ids = get_matches_from_players(summoner_ids, API_KEY, matches_per_player=10)

    # Processar estatísticas
    stats = get_champion_stats(match_ids, API_KEY)

    # Gerar relatório (mínimo 5 jogos por campeão/mapa)
    df = generate_report(stats, min_games=5)

    print(df.sort_values(by='Win Rate (%)', ascending=False))

   Champion  Map  Jogos  Win Rate (%)  Kills (avg)  Deaths (avg)  \
3      Lulu   11      6         83.33         2.17          3.83   
4     Teemo   11      5         80.00         7.00          3.20   
45    Leona   11     10         80.00         1.70          6.00   
60    Viego   11      5         80.00        13.40          5.40   
1        Vi   11     13         76.92         6.00          3.69   
..      ...  ...    ...           ...          ...           ...   
58    Garen   11      8         25.00         4.75          5.25   
47     Jinx   11      9         22.22         7.22          6.67   
50   Anivia   11      5         20.00         2.60          4.00   
28  Caitlyn   11      6         16.67         3.83          5.33   
21     Jhin   11      7         14.29         5.71          5.29   

    Assists (avg)   KDA  Gold (avg)  Dano (avg)  
3           19.83  5.74     8136.17     9546.17  
4           10.60  5.50    12128.80    27582.80  
45          16.90  3.10     8170.

In [ ]:
df

#Win Rate (%): Porcentagem de vitórias do campeão no mapa.

#KDA: Média de (kills + assists) / deaths.

#Gold (avg): Ouro médio ganho por partida.

#Dano (avg): Dano médio causado a campeões.

,Champion,Map,Jogos,Win Rate (%),Kills (avg),Deaths (avg),Assists (avg),KDA,Gold (avg),Dano (avg)
0,Ashe,11,15,66.67,7.07,5.87,10.00,2.91,11791.73,24434.87
1,Vi,11,13,76.92,6.00,3.69,8.77,4.00,10123.77,15354.38
2,Aurora,11,13,61.54,6.15,4.15,5.08,2.70,10791.92,21855.38
3,Lulu,11,6,83.33,2.17,3.83,19.83,5.74,8136.17,9546.17
4,Teemo,11,5,80.00,7.00,3.20,10.60,5.50,12128.80,27582.80
...,...,...,...,...,...,...,...,...,...,...
61,Zyra,11,7,28.57,5.86,7.71,8.43,1.85,11294.86,24010.86
62,Soraka,11,6,66.67,0.50,5.67,16.67,3.03,8023.00,5146.67
63,Hecarim,11,5,40.00,7.00,6.80,14.00,3.09,12426.80,23048.20
64,Azir,11,6,50.00,5.50,5.00,7.00,2.50,11951.50,23521.17


In [ ]:
import requests
import pandas as pd
import time

API_KEY = "RGAPI-c46c24fc-bd6b-40c2-856c-840eb0a91eea"
REGION = 'americas'
REGION_SUMMONER = 'br1'

def get_challenger_players(api_key, num_players=100):
    challenger_url = f'https://{REGION_SUMMONER}.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5'
    response = requests.get(challenger_url, headers={'X-Riot-Token': api_key})
    data = response.json()
    if 'entries' not in data:
        print("Erro ou resposta inesperada ao obter jogadores Challenger:", data)
        return []
    return [entry['summonerId'] for entry in data['entries'][:num_players]]

def get_match_details(match_id):
    url = f'https://{REGION}.api.riotgames.com/lol/match/v5/matches/{match_id}'
    headers = {"X-Riot-Token": API_KEY}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Erro ao obter detalhes da partida {match_id}: {response.text}")
        return None

def get_matches_by_map(summoner_ids, api_key, matches_per_player=20, target_matches=1000):
    map_matches = {}  # dicionário: chave -> mapId, valor -> set de match IDs

    for summoner_id in summoner_ids:
        summoner_url = f'https://{REGION_SUMMONER}.api.riotgames.com/lol/summoner/v4/summoners/{summoner_id}'
        response = requests.get(summoner_url, headers={'X-Riot-Token': api_key})
        summoner_data = response.json()
        if 'puuid' not in summoner_data:
            print("Erro ao obter puuid para summonerId:", summoner_id)
            continue
        puuid = summoner_data['puuid']

        matches_url = f'https://{REGION}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?count={matches_per_player}'
        try:
            match_ids = requests.get(matches_url, headers={'X-Riot-Token': api_key}).json()
        except Exception as e:
            print(f"Erro na requisição de match IDs para puuid {puuid}: {e}")
            continue

        for mid in match_ids:
            # Se já atingiu o target para todos os mapas conhecidos, pode interromper (opcional)
            if map_matches and all(len(match_set) >= target_matches for match_set in map_matches.values()):
                break

            match_data = get_match_details(mid)
            if not match_data or "info" not in match_data:
                continue
            map_id = match_data["info"].get("mapId")
            if map_id is None:
                continue

            if map_id not in map_matches:
                map_matches[map_id] = set()
            if mid not in map_matches[map_id]:
                map_matches[map_id].add(mid)
                print(f"Map {map_id}: {len(map_matches[map_id])} partidas acumuladas.")

            time.sleep(1.2)  # Delay para rate limits

        if map_matches and all(len(match_set) >= target_matches for match_set in map_matches.values()):
            print("Alvo atingido para todos os mapas coletados.")
            break

    for m in map_matches:
        map_matches[m] = list(map_matches[m])

    print("Total de partidas coletadas por mapa:", {k: len(v) for k, v in map_matches.items()})
    return map_matches

def get_champion_stats(match_ids_by_map, api_key):
    # Função fictícia: implemente a lógica para extrair estatísticas de cada partida
    return {}

def generate_report(champion_data, min_games=5):
    report = []

    for champion, maps in champion_data.items():
        for map_id, stats in maps.items():
            if stats['games'] < min_games:
                continue
            avg_kills = stats['kills'] / stats['games']
            avg_deaths = stats['deaths'] / stats['games']
            avg_assists = stats['assists'] / stats['games']
            avg_gold = stats['gold'] / stats['games']
            avg_damage = stats['damage'] / stats['games']
            win_rate = (stats['wins'] / stats['games']) * 100

            report.append({
                'Champion': champion,
                'Map': map_id,
                'Jogos': stats['games'],
                'Win Rate (%)': round(win_rate, 2),
                'Kills (avg)': round(avg_kills, 2),
                'Deaths (avg)': round(avg_deaths, 2),
                'Assists (avg)': round(avg_assists, 2),
                'KDA': round((avg_kills + avg_assists) / avg_deaths, 2) if avg_deaths > 0 else 'Perfect',
                'Gold (avg)': round(avg_gold, 2),
                'Dano (avg)': round(avg_damage, 2)
            })

    df_report = pd.DataFrame(report)
    return df_report

if __name__ == '__main__':
    summoner_ids = get_challenger_players(API_KEY, num_players=100)
    matches_by_map = get_matches_by_map(summoner_ids, API_KEY, matches_per_player=20, target_matches=1000)

    # Se não coletamos partidas, avisamos e paramos
    if not matches_by_map:
        print("Nenhuma partida foi coletada para nenhum mapa.")
    else:
        stats = get_champion_stats(matches_by_map, API_KEY)
        df = generate_report(stats, min_games=5)

        if df.empty:
            print("O relatório gerado está vazio. Verifique se os dados foram coletados corretamente.")
        else:
            # Somente tenta ordenar se a coluna existir
            if 'Win Rate (%)' in df.columns:
                df = df.sort_values(by='Win Rate (%)', ascending=False)
            print(df)


Map 11: 1 partidas acumuladas.
Map 11: 2 partidas acumuladas.
Map 12: 1 partidas acumuladas.
Map 11: 3 partidas acumuladas.
Map 11: 4 partidas acumuladas.
Map 11: 5 partidas acumuladas.
Map 11: 6 partidas acumuladas.
Map 11: 7 partidas acumuladas.
Map 11: 8 partidas acumuladas.
Map 11: 9 partidas acumuladas.
Map 11: 10 partidas acumuladas.
Map 11: 11 partidas acumuladas.
Map 11: 12 partidas acumuladas.
Map 11: 13 partidas acumuladas.
Map 11: 14 partidas acumuladas.
Map 12: 2 partidas acumuladas.
Map 11: 15 partidas acumuladas.
Map 11: 16 partidas acumuladas.
Map 11: 17 partidas acumuladas.
Map 11: 18 partidas acumuladas.
Map 11: 19 partidas acumuladas.
Map 11: 20 partidas acumuladas.
Map 11: 21 partidas acumuladas.
Map 11: 22 partidas acumuladas.
Map 11: 23 partidas acumuladas.
Map 11: 24 partidas acumuladas.
Map 11: 25 partidas acumuladas.
Map 11: 26 partidas acumuladas.
Map 11: 27 partidas acumuladas.
Map 11: 28 partidas acumuladas.
Map 11: 29 partidas acumuladas.
Map 11: 30 partidas